In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/Users/jacobyeung/anaconda3/envs/metaearth/lib/python3.8/site-packages')

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import shapely
import subprocess
# from metaearth.snotel import *
from metloom.pointdata.snotel_client import SeriesSnotelClient, MetaDataSnotelClient
from metloom.pointdata.snotel import SnotelPointData
from datetime import datetime
import geopandas as gpd

In [4]:
snotel_summary = pd.read_csv('../../snotel_data/SNOTEL_summary.csv', delimiter=',', 
encoding_errors="ignore", lineterminator='\n')
snotel_summary.columns = [col.rstrip(' ') for col in snotel_summary.columns]

In [5]:
snotel_summary.columns

Index(['Num', 'SNOTEL ID', 'SNOTEL State', 'SNOTEL Name', 'SNOTEL Elev (ft)',
       'SNOTEL Lat ()', 'SNOTEL Lon ()', 'Data Starting Date',
       'Data Ending Date', 'Unnamed: 9', 'Unnamed: 10', '\r'],
      dtype='object')

In [6]:
snotel_summary.head()

Num  SNOTEL ID SNOTEL State    SNOTEL Name  SNOTEL Elev (ft)  \
0    1       1000           OR  Annie Springs              6010   
1    2       1001           AK      Long Lake               850   
2    3       1003           AK  Mcneil Canyon              1320   
3    4       1005           CO      Ute Creek             10650   
4    5       1006           NV     Lewis Peak              7400   

   SNOTEL Lat ()  SNOTEL Lon () Data Starting Date Data Ending Date  \
0          42.87        -122.17          9/15/2000        9/30/2018   
1          58.19        -133.83          10/1/1999        9/30/2018   
2          59.74        -151.25          10/1/1986        9/30/2018   
3          37.61        -105.37          8/15/2000        9/30/2018   
4          40.36        -116.86          8/23/2000        9/30/2018   

   Unnamed: 9  Unnamed: 10  \r  
0         NaN          NaN  \r  
1         NaN          NaN  \r  
2         NaN          NaN  \r  
3         NaN          NaN  \r  
4         NaN          NaN  \r

In [7]:
def get_triplet(snotel_summary, id):
    curr_snotel = snotel_summary[snotel_summary['SNOTEL ID']==id]
    id = curr_snotel['SNOTEL ID'].values[0]
    state = curr_snotel['SNOTEL State'].values[0]
    triplet = f'{id}:{state}:SNTL'
    return triplet

In [8]:
rawah_triplet = get_triplet(snotel_summary, 1032)

In [9]:
rawah_triplet

'1032:CO:SNTL'

In [10]:
snotel_meta = MetaDataSnotelClient(rawah_triplet)#, begin_date='2019-01-01', end_date='2019-01-31')

In [11]:
snotel_meta.get_data()

{'actonId': '06J20S',
 'beginDate': '2002-09-01 00:00:00',
 'countyName': 'Jackson',
 'elevation': Decimal('9069.00'),
 'endDate': '2100-01-01 00:00:00',
 'fipsCountryCd': 'US',
 'fipsCountyCd': '057',
 'fipsStateNumber': '08',
 'huc': '101800010603',
 'hud': '10180001',
 'latitude': Decimal('40.70794'),
 'longitude': Decimal('-106.00727'),
 'name': 'Rawah',
 'shefId': 'RAWC2',
 'stationDataTimeZone': Decimal('-8.0'),
 'stationTimeZone': None,
 'stationTriplet': '1032:CO:SNTL'}

In [12]:
snotel_meta.SERVICE_NAME

'getStationMetadata'

# SeriesSnotelClient

In [13]:
snotel_meta = SeriesSnotelClient(station_triplet=rawah_triplet, begin_date=datetime(2019,1,1), end_date=datetime(2019,1,31))

In [14]:
len(snotel_meta.get_data("WTEQ"))

31

In [16]:
snotel_point = SnotelPointData("713:CO:SNTL", "Rawah")
df = snotel_point.get_daily_data(
    datetime(2020, 1, 2), datetime(2020, 1, 2),
    [snotel_point.ALLOWED_VARIABLES.SWE]
)
print(df)

                                                                        geometry  \
datetime                  site                                                     
2020-01-02 08:00:00+00:00 713:CO:SNTL  POINT Z (-107.71389 37.89168 11080.00000)   

                                        SWE SWE_units datasource  
datetime                  site                                    
2020-01-02 08:00:00+00:00 713:CO:SNTL  11.6        in       NRCS  


# Area of interest

In [30]:
aoi_file = '../config/aoi/aso-2013-2019-regions.json'
region = gpd.read_file(aoi_file)


In [31]:
region

geometry
0  MULTIPOLYGON (((-119.09765 38.00096, -119.0976...

In [32]:
region.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [57]:
snotel_point = SnotelPointData("713:CO:SNTL", "MyStation")
vars = [snotel_point.ALLOWED_VARIABLES.SWE]
points = SnotelPointData.points_from_geometry(region, vars)
df = points.to_dataframe()
print(df)

Empty GeoDataFrame
Columns: [name, id, datasource, geometry]
Index: []


In [23]:
region.crs

AttributeError: 'Polygon' object has no attribute 'crs'